In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")

Using TensorFlow backend.


## Dataset Pre-processing

In [73]:
TASKS = ["climbing","jumping","running"]
LABELS = {
    "climbing" : 1,
    "jumping" : 2,
    "running" : 3,
}
SENSORS = ["Accelerometer","Gyroscope"]
dataset_path = "dataset/train/"

In [83]:
data = {}
for sensors in SENSORS:
    data[sensors] = []
for vals in TASKS:
    sub_dirs = [x for x in os.walk(dataset_path + vals)]
    datapoints = sub_dirs[0][1]
    for pts in datapoints:
        for sensors in SENSORS:
            file_path = dataset_path + vals + "/" + pts + "/" +sensors + ".csv"
            df = pd.read_csv(file_path)
            df["Label"] = LABELS[vals]
            data[sensors].append(df)

In [84]:
def build_model(x_train, y_train):
    
    n_time_steps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    
    verbose, epochs, batch_size = 0, 70, 16
    
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')

    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
